In [1]:
# !conda install tensorflow-gpu=1.15.0 keras=2.2.4 bcolz h5py matplotlib=2.1.1 mkl nose numpy=1.13.3 scipy rasterio pydot pygpu pyyaml six

In [2]:
from __future__ import division

import random
import h5py
import keras.backend as K
import numpy as np
import tensorflow as tf
import tools.image_metrics as img_met
from dsen2cr_network import DSen2CR_model
from dsen2cr_tools import train_dsen2cr, predict_dsen2cr
from keras.optimizers import Nadam
from keras.utils import multi_gpu_model
from tools.dataIO import get_train_val_test_filelists, process_predicted, DataGenerator, get_preview
from data import get_train_val_test_split
from dsen2cr_tools import make_dir
import csv
import os
from tqdm import tqdm
import tifffile
import matplotlib.pyplot as plt 

K.set_image_data_format("channels_first")

print('version', tf.version.VERSION)

Using TensorFlow backend.


version 1.15.0


In [3]:
f = h5py.File("./out/model_SARcarl.hdf5", 'r')
list(f.keys())

['activation_1',
 'activation_10',
 'activation_11',
 'activation_12',
 'activation_13',
 'activation_14',
 'activation_15',
 'activation_16',
 'activation_17',
 'activation_2',
 'activation_3',
 'activation_4',
 'activation_5',
 'activation_6',
 'activation_7',
 'activation_8',
 'activation_9',
 'add_1',
 'add_10',
 'add_11',
 'add_12',
 'add_13',
 'add_14',
 'add_15',
 'add_16',
 'add_17',
 'add_2',
 'add_3',
 'add_4',
 'add_5',
 'add_6',
 'add_7',
 'add_8',
 'add_9',
 'concatenate_1',
 'concatenate_2',
 'conv2d_1',
 'conv2d_10',
 'conv2d_11',
 'conv2d_12',
 'conv2d_13',
 'conv2d_14',
 'conv2d_15',
 'conv2d_16',
 'conv2d_17',
 'conv2d_18',
 'conv2d_19',
 'conv2d_2',
 'conv2d_20',
 'conv2d_21',
 'conv2d_22',
 'conv2d_23',
 'conv2d_24',
 'conv2d_25',
 'conv2d_26',
 'conv2d_27',
 'conv2d_28',
 'conv2d_29',
 'conv2d_3',
 'conv2d_30',
 'conv2d_31',
 'conv2d_32',
 'conv2d_33',
 'conv2d_34',
 'conv2d_4',
 'conv2d_5',
 'conv2d_6',
 'conv2d_7',
 'conv2d_8',
 'conv2d_9',
 'input_1',
 'input_2'

In [4]:
model_name = "DSen2-CR_001"  # model name for training

# model parameters
num_layers = 16  # B value in paper
feature_size = 256  # F value in paper

# include the SAR layers as input to model
include_sar_input = True

# cloud mask parameters
use_cloud_mask = True
cloud_threshold = 0.2  # set threshold for binarisation of cloud mask

# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Setup data processing param %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# input data preprocessing parameters
scale = 2000
max_val_sar = 2
clip_min = [
    [-25.0, -32.5],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
]
clip_max = [
    [0, 0],
    [
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
    ],
    [
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
        10000,
    ],
]

shuffle_train = True  # shuffle images at training time
data_augmentation = True  # flip and rotate images randomly for data augmentation

random_crop = True  # crop out a part of the input image randomly
crop_size = 128  # crop size for training images

In [5]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Setup training %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# dataset_list_filepath = "./gen-cloudless/datasetfilelist.csv"

base_out_path = "./out/"
input_data_folder = "./data/"

# training parameters
initial_epoch = 0  # start at epoch number
epochs_nr = 8  # train for this amount of epochs. Checkpoints will be generated at the end of each epoch
batch_size = 16  # training batch size to distribute over GPUs

# define metric to be optimized
loss = img_met.carl_error
# define metrics to monitor
metrics = [
    img_met.carl_error,
    img_met.cloud_mean_absolute_error,
    img_met.cloud_mean_squared_error,
    img_met.cloud_mean_sam,
    img_met.cloud_mean_absolute_error_clear,
    img_met.cloud_psnr,
    img_met.cloud_root_mean_squared_error,
    img_met.cloud_bandwise_root_mean_squared_error,
    img_met.cloud_mean_absolute_error_covered,
    img_met.cloud_ssim,
    img_met.cloud_mean_sam_covered,
    img_met.cloud_mean_sam_clear,
]

# define learning rate
lr = 7e-5

# initialize optimizer
optimizer = Nadam(
    lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-8, schedule_decay=0.004
)

In [6]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Other setup parameters %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

predict_data_type = "test"  # possible options: 'val' or 'test'

log_step_freq = 1  # frequency of logging

n_gpus = 1  # set number of GPUs
# multiprocessing optimization setup
use_multi_processing = True
max_queue_size = 1 # 2 * n_gpus
workers = 1 # 4 * n_gpus

batch_per_gpu = int(batch_size / n_gpus)

input_shape = ((13, crop_size, crop_size), (2, crop_size, crop_size))

In [7]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Initialize session %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# Configure Tensorflow session
config = tf.ConfigProto()
# Don't pre-allocate memory; allocate as-needed
config.gpu_options.allow_growth = True

# Only allow a total % of the GPU memory to be allocated
# config.gpu_options.per_process_gpu_memory_fraction = 0.3

# Create a session with the above options specified.
K.tensorflow_backend.set_session(tf.Session(config=config))

# Set random seeds for repeatability
random_seed_general = 42
random.seed(random_seed_general)  # random package
np.random.seed(random_seed_general)  # numpy package
tf.set_random_seed(random_seed_general)  # tensorflow

2022-01-11 19:50:18.634197: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-01-11 19:50:18.643394: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 3000040000 Hz
2022-01-11 19:50:18.646918: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55ef17a50d90 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-01-11 19:50:18.647002: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-01-11 19:50:18.652639: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2022-01-11 19:50:18.821079: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-01-11 19:50:18.821

In [8]:
# %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%% Initialize model %%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%

# single or no-gpu case
if n_gpus <= 1:
    model, shape_n = DSen2CR_model(
        input_shape,
        batch_per_gpu=batch_per_gpu,
        num_layers=num_layers,
        feature_size=feature_size,
        use_cloud_mask=use_cloud_mask,
        include_sar_input=include_sar_input,
    )
else:
    # handle multiple gpus
    with tf.device("/cpu:0"):
        single_model, shape_n = DSen2CR_model(
            input_shape,
            batch_per_gpu=batch_per_gpu,
            num_layers=num_layers,
            feature_size=feature_size,
            use_cloud_mask=use_cloud_mask,
            include_sar_input=include_sar_input,
        )

    model = multi_gpu_model(single_model, gpus=n_gpus)

model.compile(optimizer=optimizer, loss=loss, metrics=metrics)
print("Model compiled successfully!")

print("Getting file lists")
# train_filelist, val_filelist, test_filelist = get_train_val_test_filelists(
#     dataset_list_filepath
# )
train_filelist, val_filelist, test_filelist = get_train_val_test_split()

print("Number of train files found: ", len(train_filelist))
print("Number of validation files found: ", len(val_filelist))
print("Number of test files found: ", len(test_filelist))









Instructions for updating:
Deprecated in favor of operator or tf.math.divide.

Model compiled successfully!
Getting file lists
Number of train files found:  27061
Number of validation files found:  3383
Number of test files found:  3383


In [9]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 13, 128, 128) 0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 2, 128, 128)  0                                            
__________________________________________________________________________________________________
concatenate_1 (Concatenate)     (None, 15, 128, 128) 0           input_1[0][0]                    
                                                                 input_2[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 256, 128, 128 34816       concatenate_1[0][0]              
__________

In [11]:
predict_file = "./out/model_SARcarl.hdf5"
print("Predicting using file: {}".format(predict_file))
print("Using this model: {}".format(model_name))

# load the model weights at checkpoint
model.load_weights(predict_file)

out_path_predict = make_dir(os.path.join(base_out_path, model_name))
predicted_images_path = make_dir(os.path.join(out_path_predict, "predictions/"))

print("Initializing generator for prediction and evaluation")
params = {
    "input_dim": input_shape,
    "batch_size": 16,
    "shuffle": False,
    "scale": scale,
    "include_target": True,
    "data_augmentation": False,
    "random_crop": False,
    "crop_size": crop_size,
    "clip_min": clip_min,
    "clip_max": clip_max,
    "input_data_folder": input_data_folder,
    "use_cloud_mask": use_cloud_mask,
    "max_val_sar": max_val_sar,
    "cloud_threshold": cloud_threshold,
}
predict_generator = DataGenerator(test_filelist, **params)

eval_csv_name = out_path_predict + "eval.csv"
print("Storing evaluation metrics at ", eval_csv_name)


def batch_save(predictions, filenames, out_path):
    for i in range(len(predictions)):
        tifffile.imsave(os.path.join(out_path, filenames[i]), predictions[i])

        img = get_preview(predictions[i], True, [4, 3, 2], 2000)
        plt.figure(frameon=False)
        plt.imshow(img)
        plt.gca().get_xaxis().set_visible(False)
        plt.gca().get_yaxis().set_visible(False)
        plt.axis('off')
        plt.savefig(os.path.join(out_path, filenames[i] + '.png'), dpi=600, bbox_inches='tight', pad_inches=0)
        plt.close()


for i, (data, y) in enumerate(tqdm(predict_generator)):
    # predict output image
    predicted = model.predict_on_batch(data)
    # process predicted image

    batch_save(predicted, test_filelist[i * batch_size : i * batch_size + batch_size], predicted_images_path)

print("Prediction finished with success!")

Predicting using file: ./out/model_SARcarl.hdf5
Using this model: DSen2-CR_001
Initializing generator for prediction and evaluation
Generator initialized
Storing evaluation metrics at  ./out/DSen2-CR_001eval.csv


  0%|          | 0/211 [00:59<?, ?it/s]

Prediction finished with success!


In [ ]:
train_filelist, val_filelist, test_filelist = get_train_val_test_split()

params = {
    "input_dim": input_shape,
    "batch_size": 1,
    "shuffle": False,
    "scale": scale,
    "include_target": True,
    "data_augmentation": False,
    "random_crop": False,
    "crop_size": crop_size,
    "clip_min": clip_min,
    "clip_max": clip_max,
    "input_data_folder": input_data_folder,
    "use_cloud_mask": use_cloud_mask,
    "max_val_sar": max_val_sar,
    "cloud_threshold": cloud_threshold,
}

predict_generator = DataGenerator(test_filelist, **params)
(X, sar), y = predict_generator.__getitem__(0)


Generator initialized


In [ ]:
X.shape

(1, 13, 128, 128)

In [ ]:
sar.shape

(1, 2, 128, 128)

In [ ]:
y[0].shape

(1, 14, 128, 128)

In [ ]:
for i, (data, y) in enumerate(predict_generator):
  X, sar = data
  print(X.shape, sar.shape, y[0].shape)
  break

(1, 13, 128, 128) (1, 2, 128, 128) (1, 14, 128, 128)
